In [10]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [11]:
#upload data into dataframes
triplet_data = pd.read_table('/Users/christinacampbell/downloads/10000.txt',header = None)
triplet_data.columns= ['user_id', 'song_id', 'listen_count']
song_data = pd.read_csv('/Users/christinacampbell/downloads/song_data.csv')
triplet_data=triplet_data.sort_values(by='song_id').reindex()
song_data=song_data.sort_values(by='song_id').reindex()

In [21]:
#use triplet data to train
triplet_data=triplet_data.head(100000)
triplet_data.head()

,user_id,song_id,listen_count
639951,8fffc01119b0f79c95cad10b5827f9a60d19a68f,SOAAAGQ12A8C1420C8,1
848788,2c2a59d10d1c17b766567f0cde9955571545e710,SOAAAGQ12A8C1420C8,5
1507440,3192565c00ec4e9b91e90d5bce979c319a963721,SOAAAGQ12A8C1420C8,35
67468,43c6db7ff4608324722a10cc95bd1c5c69ab7952,SOAAAGQ12A8C1420C8,1
1097056,be82636cdaa522689141f43d3c97d807e8d34f30,SOAAAGQ12A8C1420C8,4


In [31]:
#use song data to test
song_data=song_data.head(100000)
song_data.head(700)

,song_id,title,release,artist_name,year
234946,SOAAABI12A8C13615F,Afro Jazziac,To Birdland And Hurry,Herbie Mann,2000
21668,SOAAABT12AC46860F0,Herre Gud Ditt Dyre Namn Og Ære,Som Den Gyldne Sol Frembryter,Bergen Big Band,0
877870,SOAAABX12A8C13FEB2,N.Y.C. Remix,Paris Can´t Wait,Guardner,0
831028,SOAAACR12A58A79456,Irresistible,Wowie Zowie,Superchumbo,2002
882278,SOAAACY12A58A79663,Untitled 1,Pine Cone Temples,Thuja,0
469097,SOAAADD12AB018A9DD,The Room Of Stairs,The Room,Harold Budd,2000
767863,SOAAADE12A6D4F80CC,(I'm Gonna Start) Living Again If It Kills Me ...,Twangin',Dave Edmunds,1981
165522,SOAAADF12A8C13DF62,KRYSAR (LIVE),Vltava Tour,LANDA DANIEL,0
437759,SOAAADP12A8C1413C7,1-2 (Victory Check),Victory Live,Tye Tribbett & G.A.,2006
32000,SOAAADZ12A8C1334FB,Joy To The World,The Christmas Box,Paul Cardall,1997


In [15]:
song_features = triplet_data.pivot(
    index='song_id',
    columns='user_id',
    values='listen_count'
).fillna(0)

In [17]:
matrix_song_features = csr_matrix(song_features.values)

In [19]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
model_knn.fit(matrix_song_features)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=-1, n_neighbors=20, p=2, radius=1.0)

In [24]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
def print_artist_recommendations(query_song, song_matrix, knn_model, k):

    query_index = None
    ratio_tuples = []
    
    for i in song_matrix.index:
        ratio = fuzz.ratio(i.lower(), query_song.lower())
        if ratio >= 75:
            current_query_index = song_matrix.index.tolist().index(i)
            ratio_tuples.append((i, ratio, current_query_index))
    
    print ('Number of possible matches: {0}\n'.format([(x[0], x[1]) for x in ratio_tuples]))
    
    try:
        query_index = max(ratio_tuples, key = lambda x: x[1])[2] # get the index of the best artist match in the data
    except:
        print ('Artist not found in the data. Try again')
        return None
    
    distances, indices = knn_model.kneighbors(song_matrix.iloc[query_index, :].values.reshape(1, -1), n_neighbors = k + 1)

    for i in range(0, len(distances.flatten())):
        if i == 0:
            print ('Recommendations for {0}:\n'.format(song_matrix.index[query_index]))
        else:
            print ('{0}: {1}, with distance of {2}:'.format(i, song_matrix.index[indices.flatten()[i]], distances.flatten()[i]))

    return None

/Users/christinacampbell/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [29]:
#use a song_ID from triplet data (test data)
#need to change it so that it lists the song instead  of the song id
# need to limit the size of song_data so that there are less songs not in the training
print_artist_recommendations("SOAAAGQ12A8C1420C8", song_features, model_knn, 2)

Number of possible matches: [('SOAAAGQ12A8C1420C8', 100)]

Recommendations for SOAAAGQ12A8C1420C8:

1: SOACERJ12A67AD865E, with distance of 0.9366308219305053:
2: SOATKQE12A8C133A97, with distance of 0.9451998547827779:
